In [1]:
from optimizers import OptimizationTask, OptimizationResult, Optimizer, optimizers
from dataset import Dataset, Instance
from pennylane import QNode
import pennylane.numpy as np
from math import pi

In [2]:
def generate_tasks(
    circuit_id: str,
    num_qubits: int,
    num_layers: int,
    rp_params_shape: tuple,
    crp_params_shape: tuple,
    max_evaluations: int = 250,
    convergence_threshold: int = 1e-6,
    num_tasks: int = 100
) -> list[OptimizationTask]:
    return [
        OptimizationTask(
            circuit_id=circuit_id,
            num_qubits=num_qubits,
            num_layers=num_layers,
            initial_params=(
                np.random.random(rp_params_shape) * 2 * pi,
                np.random.random(crp_params_shape) * 4 * pi
            ),
            max_evaluations=max_evaluations,
            convergence_threshold=convergence_threshold
        )
        for _ in range(num_tasks)
    ]

In [3]:
def run_task(
        task: OptimizationTask,
        optimizers: list[tuple[str, Optimizer]]
) -> Instance:
    print(f"Starting optimization for {task.circuit_id}_{task.num_qubits}x{task.num_layers}...")
    results: dict[str, OptimizationResult] = {
        name: optimize(task)
        for name, optimize in optimizers
    }
    
    return Instance(
        task=task,
        results=results,
    )

In [4]:
from circuits import sim_03

num_qubits=4
num_layers=3

common_params = {
    "num_qubits": num_qubits,
    "num_layers": num_layers,
    "num_tasks": 100,
    "max_evaluations": 250
}

tasks = []

tasks.extend(generate_tasks(
    circuit_id="sim01",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = 0,
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim02",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = 0,
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim03",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim04",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim05",
    rp_params_shape = (num_layers, num_qubits, 4),
    crp_params_shape = (num_layers, num_qubits, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim06",
    rp_params_shape = (num_layers, num_qubits, 4),
    crp_params_shape = (num_layers, num_qubits, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim07",
    rp_params_shape = (num_layers, num_qubits, 4),
    crp_params_shape = (num_layers, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim08",
    rp_params_shape = (num_layers, num_qubits, 4),
    crp_params_shape = (num_layers, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim09",
    rp_params_shape = (num_layers, num_qubits),
    crp_params_shape = 0,
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim10",
    rp_params_shape = (num_layers + 1, num_qubits),
    crp_params_shape = 0,
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim11",
    rp_params_shape = (num_layers, num_qubits, 3),
    crp_params_shape = 0,
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim12",
    rp_params_shape = (num_layers, num_qubits, 3),
    crp_params_shape = 0,
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim13",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits, 2),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim14",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits, 2),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim15",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = 0,
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim16",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim17",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits - 1),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim18",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits),
    **common_params
))
tasks.extend(generate_tasks(
    circuit_id="sim19",
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits),
    **common_params
))

In [5]:
%%time
from concurrent.futures import Future, ProcessPoolExecutor, as_completed
from loky.backend.context import cpu_count
import time

cores = cpu_count()
print(f"Running with {cores} cores.")

folder = "../dataset/"

with ProcessPoolExecutor(max_workers=cores) as executor:
    task_map: dict[Future[Instance], OptimizationTask] = {
        executor.submit(run_task, task, optimizers): task
        for task in tasks
    }

    completed = 0
    for future in as_completed(task_map):
        instance: Instance = future.result()
        instance.save(folder)
        completed += 1

        task = task_map[future]
        print(f"[{completed}/{len(tasks)}] {time.asctime()} - Completed {task.circuit_id}_{task.num_qubits}x{task.num_layers} instance.")



Running with 8 cores.


Starting optimization for sim01_4x3...Starting optimization for sim04_4x3...Starting optimization for sim02_4x3...Starting optimization for sim03_4x3...Starting optimization for sim05_4x3...


Starting optimization for sim08_4x3...

Starting optimization for sim07_4x3...Starting optimization for sim06_4x3...




/home/max/Projects/bachelor-thesis/.venv/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")
/home/max/Projects/bachelor-thesis/.venv/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Starting optimization for sim09_4x3...
[1/19] Mon Dec  4 00:10:20 2023 - Completed sim01_4x3 instance.
Starting optimization for sim10_4x3...
[2/19] Mon Dec  4 00:10:22 2023 - Completed sim09_4x3 instance.
Starting optimization for sim11_4x3...
[3/19] Mon Dec  4 00:10:24 2023 - Completed sim02_4x3 instance.
Starting optimization for sim12_4x3...
[4/19] Mon Dec  4 00:10:31 2023 - Completed sim10_4x3 instance.
Starting optimization for sim13_4x3...
[5/19] Mon Dec  4 00:10:33 2023 - Completed sim03_4x3 instance.
Starting optimization for sim14_4x3...
[6/19] Mon Dec  4 00:10:40 2023 - Completed sim04_4x3 instance.
Starting optimization for sim15_4x3...
[7/19] Mon Dec  4 00:10:53 2023 - Completed sim11_4x3 instance.
Starting optimization for sim16_4x3...
[8/19] Mon Dec  4 00:11:07 2023 - Completed sim12_4x3 instance.
Starting optimization for sim17_4x3...
[9/19] Mon Dec  4 00:11:12 2023 - Completed sim15_4x3 instance.
Starting optimization for sim18_4x3...
[10/19] Mon Dec  4 00:11:15 2023 -